In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from Tree import Tree
import matplotlib.pyplot as plt
from tqdm import tqdm

In [43]:
df = pd.read_csv('ufo.csv')
df.drop('datetime', axis=1, inplace=True)
df.drop('city', axis=1, inplace=True)
df.drop('state', axis=1, inplace=True)
df.drop('country', axis=1, inplace=True)
df.drop('duration (hours/min)', axis=1, inplace=True)
df.drop('comments', axis=1, inplace=True)
df.drop('date posted', axis=1, inplace=True)
df.rename({'duration (seconds)': 'duration'}, axis=1, inplace=True)
df.rename({'longitude ': 'longitude'}, axis=1, inplace=True)
def floatify(t):
    try:
        return float(t)
    except:
        return 0
df['duration'] = df['duration'].apply(floatify)
df['latitude'] = df['latitude'].apply(floatify)
df['longitude'] = df['longitude'].apply(floatify)
df.drop(df[df['duration'] > 1800].index, inplace=True)
df.drop(df[df['duration'] == 0].index, inplace=True)
df.drop(df[df['longitude'] == 0].index, inplace=True)
df.drop(df[df['latitude'] == 0].index, inplace=True)
df.dropna(inplace=True)
df.head()

,shape,duration,latitude,longitude
2,circle,20.0,53.200000,-2.916667
3,circle,20.0,28.978333,-96.645833
4,light,900.0,21.418056,-157.803611
5,sphere,300.0,36.595000,-82.188889
6,circle,180.0,51.434722,-3.180000


In [46]:
train = df.sample(frac=0.8)
test  = df.drop(train.index)

train_shape = train.rename(columns={'shape':'target'})
test_shape  =  test.rename(columns={'shape':'target'})

X_train_duration = train.drop('duration', axis=1)
X_test_duration  =  test.drop('duration', axis=1)
Y_train_duration = train.duration
Y_test_duration  =  test.duration

In [33]:
tree = Tree(train_shape)
predictions = test_shape.apply(tree.classify, axis=1)
correct = predictions == test_shape.target
accuracy = correct.value_counts()[True]/len(correct)
targets = df['shape'].value_counts().keys()
records = {}
print(f"shape: {accuracy} on {len(targets)} targets")
for target in targets:
    test_target = test_shape[test_shape.target==target]
    if len(test_target) < 5:
        break
    predictions_target = predictions[test_target.index]
    correct = predictions_target == test_target.target
    vc = correct.value_counts()
    if True in vc:
        accuracy = vc[True]/len(correct)
    else:
        accuracy = 0
    records[target] = accuracy
    print(f"    {target}: {accuracy}")


shape: 0.14458844133099824 on 28 targets
    light: 0.4246668944311582
    triangle: 0.12318339100346021
    circle: 0.07952146375791697
    fireball: 0.1297071129707113
    other: 0.054380664652567974
    unknown: 0.06631578947368422
    sphere: 0.06762295081967214
    disk: 0.06857727737973388
    oval: 0.058646616541353384
    formation: 0.036036036036036036
    cigar: 0.04477611940298507
    changing: 0.046511627906976744
    rectangle: 0.01606425702811245
    flash: 0.01984126984126984
    cylinder: 0.0311284046692607
    diamond: 0.03827751196172249
    chevron: 0.03468208092485549
    teardrop: 0.00684931506849315
    egg: 0.027586206896551724
    cone: 0
    cross: 0


In [35]:
import json
with open('ufo_results.json', 'w') as f:
    json.dump(records, f)

In [50]:
tree = DecisionTreeRegressor()
# X_train_duration.drop('shape', axis=1, inplace=True)
# X_test_duration.drop('shape', axis=1, inplace=True)
tree.fit(X_train_duration, Y_train_duration)
print(tree.score(X_test_duration, Y_test_duration))

-0.2977755000745772


In [51]:
Y_predict = tree.predict(X_test_duration)
residuals = Y_predict-Y_test_duration
mean_error = residuals.abs().mean()

In [52]:
mean_error

356.3819950455579